In [1]:
%load_ext autotime

In [2]:
import pymongo

mongo = pymongo.MongoClient(port=27018)
db = mongo['nlp']
collection = db['github-bq']

time: 25 ms


In [3]:
import re
import pandas as pd

time: 370 ms


In [4]:
import gensim
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel

time: 2.11 s


In [5]:
lda = LdaModel.load('../../MODELS.staging/LDA/WB-ALL_100/data/WB_lda_model_ALL_100.mallet.gensim.lda')

time: 97.3 ms


In [ ]:
dictionary = Dictionary()
dictionary.id2token = lda.id2word
dictionary.token2id = {j: i for i, j in dictionary.id2token.items()}

In [36]:
%%time
# keywords = ['economic', 'nutrition', 'income inequality', 'agriculture', 'climate change', 'poverty', 'fragility', 'refugee']
keywords = ['poverty', 'climate change', 'refugee', 'fragility', 'agriculture']
payloads = []
keyword_topic = {}

for kw in keywords:
    for data in collection.find({'$text': {'$search': f'"{kw}"'}, 'readme': {'$exists': True}}):
        payloads.append(data['readme'])
        
    for i in range(lda.num_topics):
        k = kw.replace(' ', '_')
        if k in lda.print_topic(i):
            print(f'Keyword: {k}\tTopic: {i}')
            keyword_topic.setdefault(k, []).append(i)

Keyword: poverty	Topic: 60
Keyword: poverty	Topic: 94
Keyword: climate_change	Topic: 16
Keyword: refugee	Topic: 44
Keyword: agriculture	Topic: 19
CPU times: user 10.2 s, sys: 85.5 ms, total: 10.3 s
Wall time: 12.3 s
time: 12.3 s


In [39]:
topic_targets = [i for v in keyword_topic.values() for i in v]

time: 547 µs


In [40]:
len(payloads)

423

time: 1.89 ms


In [47]:
relevant_payloads = []

for payload in payloads:
    bow = dictionary.doc2bow(re.findall('[a-z]+', payload.lower()))

    d = pd.DataFrame(lda[bow], columns=['topic', 'score'])
    d = d.sort_values('score', ascending=False).head(10)
    if set(d['topic']).intersection(topic_targets):
        relevant_payloads.append(payload)

time: 6.83 s


In [48]:
len(relevant_payloads)

244

time: 1.62 ms


In [74]:
print(relevant_payloads[25])

# Synopsis
CO-NCRN is the University of Colorado at Boulder branch of the [NSF-Census Research Network](https://www.ncrn.info/node/university-colorado-boulder-university-tennessee). The ACS Regionalization code is one project to emerge from our work, providing an algorithm that takes in small-area American Community Survey estimates and produces optimized regions that reduce high levels of uncertainty in the data. The results are maps and tables with new geographies that maintain granularity but improve data quality, thereby enhancing researchers' ability to ensure their results are not comprised by high uncertainty.

Please visit the main page of this repository, [co-ncrn.github.io](co-ncrn.github.io) to acess information about this project and obtain regionalization results for a selection of input data applied to all metropolitan statistical areas in the United States. To use your own input variables and geographies, you can access the regionalization algorithm at [this repository](

In [27]:
p = 3
bow = dictionary.doc2bow(re.findall('[a-z]+', payloads[p].lower()))

time: 1.02 ms


set()

time: 2.34 ms


In [29]:
lda.print_topic(35)

'0.070*"datum" + 0.035*"survey" + 0.023*"report" + 0.020*"information" + 0.016*"table" + 0.016*"indicator" + 0.015*"number" + 0.013*"source" + 0.013*"figure" + 0.013*"analysis"'

time: 24.5 ms


In [30]:
print(payloads[p])

# Climate change and conflict
## Isaac Menninga, Nov. 2015, Quest University Canada

## Usage: 
bash shell.sh
Script file is contained in the bin directory, outputs are saved in results directory.

##Ten Minute Plan

Climate change has many effects on natural systems. Rainfall, humidity, temperature and wind patterns will drastically change into the future. Each of these can affect a variety of things such as crop yield, human health, and infrastructure integrity. These effects will be felt soonest, and in the most extremes in developing countries, where people do not have access to the same infrastructure or technology for crisis response. As the effects of climate change continue to escalate in severity, billions of people may be displaced or unable to cope with the severe changes. The goal of the study is to determine if there is a relationship between climate change and the occurrence of conflict in developing countries. 

The code analyzes data about climate change and conflict in

In [36]:
# [i for i in re.findall('[a-z]+', payloads[4].lower()) if i in dictionary.token2id]

['refugee',
 'data',
 'analysis',
 'repository',
 'data',
 'analysis',
 'supporting',
 'news',
 'article',
 'come',
 'go',
 'in',
 'com',
 'come',
 'go',
 'in',
 'published',
 'data',
 'refugee',
 'data',
 'department',
 'state',
 'refugee',
 'processing',
 'center',
 'org',
 'data',
 'portal',
 'org',
 'default',
 'specifically',
 'destination',
 'nationality',
 'nationality',
 'religion',
 'generated',
 'raw',
 'can',
 'in',
 'repository',
 'data',
 'directory',
 'data',
 'calculate',
 'state',
 'analysis',
 'census',
 'bureau',
 'population',
 'census',
 'gov',
 'data',
 'state',
 'index',
 'analysis',
 'code',
 'analysis',
 'written',
 'in',
 'python',
 'can',
 'refugee',
 'analysis',
 'run',
 'analysis',
 'you',
 'need',
 'install',
 'python',
 'listed',
 'in',
 'alternatively',
 'you',
 'can',
 'run',
 'analysis',
 'in',
 'browser',
 'binder',
 'org',
 'in',
 'united',
 'installation',
 'required',
 'feedback',
 'email',
 'author',
 'singer',
 'vine',
 'singer',
 'vine',
 'com']

time: 2.65 ms
